# Open Data Loop

Captures, in an open loop, camera data.

Modeled off of: https://sfeister.github.io/sidekick-epics-docs/demos/OpenLoopHRR/

## Import Modules

In [3]:
import numpy as np
import epics # 'epics' is part of pyepics
from epics import PV
from time import sleep
import matplotlib.pyplot as plt

In [4]:
pvroi1 = PV("CAM:ROI1:DATA.VAL", form="time")
pvroi2 = PV("CAM:ROI2:DATA.VAL", form="time")
leds = PV("LEDS:data.VAL", form="time")

In [5]:
print(pvroi1.value)
print(pvroi2.value)
print(leds.value)

TRIG:3699,R:74,G:98,B:223
TRIG:3699,R:82,G:25,B:30
4450,255,255,255,255,26,


In [100]:
epics.caput("CAM:TRIGCNT", 12555)

1

In [45]:
def ledparse(datastr):
    """ Parse the LED brightness data string, which comes in the format '123441,251,12,201,12,156,' where it goes 'TRIG, LED0, LED1, LED2, LED3, LED4' """
    vals = np.array(datastr.split(','))[:-1]
    trig = np.int64(vals[0])
    brigs = np.array(np.uint8(vals[1:]))
    return trig, brigs

In [103]:
def camparse(datastr):
    """ Parse the RGB-color data string, which comes in the format 'TRIG:123441,R:251,G:12,B:201' """
    vals = [elem.split(":")[1] for elem in datastr.split(',')]
    trig = np.int64(vals[0])
    rgb = np.array(np.uint8(vals[1:]))
    return trig + 752, rgb

In [104]:
print(camparse(pvroi1.value))
print(camparse(pvroi2.value))
print(ledparse(leds.value))

(2207, array([ 70,  91, 240], dtype=uint8))
(2207, array([ 57,  46, 128], dtype=uint8))
(2208, array([255, 255, 255, 255, 255], dtype=uint8))


In [108]:
def dataCallback(pvname=None, value=None, **kw):
    global isfresh
    isfresh = True

epics.caput("PULSEGEN:reprate", 0.25)
pvroi1.add_callback(dataCallback)

NSTEPS = 10
for i in range(NSTEPS):
    isfresh = False
    while not isfresh:
        sleep(0.01)
    trig1, rgb1 = camparse(pvroi1.value)
    trig2, rgb2 = camparse(pvroi2.value)
    trig3, brigs = ledparse(leds.value)
    print(trig1, trig2, trig3)
    print(rgb1, rgb2, brigs)
    for j in range(len(brigs)):
        epics.caput('LEDS:CH{}:brig.VAL'.format(j), i*50)
    #print('ROI1: ' + pvroi1.value)
    #print('ROI2: ' + pvroi2.value)

pvroi1.clear_callbacks()

2270 2270 2270
[  8  71 216] [ 57  46 139] [  0  50 100 150 200]
2271 2271 2271
[36 43 46] [ 57  46 152] [  0  50 100 150 200]
2272 2272 2272
[ 44  55 146] [ 56  47 151] [0 0 0 0 0]
2273 2272 2273
[ 51  66 194] [ 56  47 151] [50 50 50 50 50]
2274 2274 2274
[ 58  76 217] [ 57  46 141] [100 100 100 100 100]
2275 2275 2275
[ 65  84 229] [ 56  46 132] [150 150 150 150 150]
2275 2275 2276
[ 65  84 229] [ 56  46 132] [200 200 200 200 200]
2277 2277 2277
[ 72  94 243] [ 56  46 132] [250 250 250 250 250]
2278 2278 2278
[ 71  90 239] [ 56  47 129] [255 255 255 255 255]
2279 2279 2279
[ 72 100 245] [ 57  51 141] [255 255 255 255 255]


In [ ]:
print(pvroi1.value)

In [9]:
pvroi1.clear_callbacks()

In [11]:
def testCallback(pvname=None, value=None, **kw):
    print('New PVROI Data!')
    print(value)

pvroi1.add_callback(testCallback)
sleep(10)
pvroi1.clear_callbacks()

New Photodetector Data!
TRIG:1828566,R:54,G:49,B:48
New Photodetector Data!
TRIG:1828567,R:54,G:49,B:48
New Photodetector Data!
TRIG:1828568,R:54,G:49,B:48
New Photodetector Data!
TRIG:1828569,R:55,G:49,B:48
New Photodetector Data!
TRIG:1828570,R:55,G:49,B:47
New Photodetector Data!
TRIG:1828571,R:55,G:49,B:48
New Photodetector Data!
TRIG:1828572,R:109,G:114,B:107
New Photodetector Data!
TRIG:1828573,R:24,G:25,B:23
New Photodetector Data!
TRIG:1828574,R:27,G:27,B:26
